# Flash Report — Operation 1 Complete (Part 3: Reports)

**Run Parts 1 and 2 first, then continue with this notebook.**

This generates Excel reports and program summaries from Operation 1 results.

## Program Summary Generation

In [ ]:
def create_program_summary(df, region_name, currency_cols):
    """Create program summary from processed data"""
    print(f'[*] Creating {region_name} Program Summary...')
    
    # Group by Scheme_Name and sum relevant columns
    summary_cols = ['Scheme_Name'] + currency_cols
    available_cols = [col for col in summary_cols if col in df.columns]
    
    if 'Scheme_Name' not in df.columns:
        print(f'[WARNING] Scheme_Name column not found in {region_name} data')
        return pd.DataFrame()
    
    program_summary = df.groupby('Scheme_Name')[currency_cols].sum().reset_index()
    program_summary = program_summary[program_summary['Scheme_Name'] != '']
    
    print(f'[*] {region_name} Program Summary created with {len(program_summary)} schemes')
    return program_summary

# Create program summaries
us_currency_cols = ['NDP_TOTAL_USD', 'NET_TOTAL_USD', 'UPFRONT_DISCOUNT_AMT_USD', 'BACKEND_DISCOUNT_AMT_USD']
ca_currency_cols = ['NDP_TOTAL_LC', 'NET_TOTAL_LC', 'UPFRONT_DISCOUNT_AMT_LC', 'BACKEND_DISCOUNT_AMT_LC']

program_summary_us = create_program_summary(report_us, 'US', us_currency_cols)
program_summary_ca = create_program_summary(report_ca, 'CA', ca_currency_cols)

print('Program summaries created successfully!')

## Excel Report Generation

In [ ]:
print('='*60)
print('GENERATING EXCEL REPORTS')
print('='*60)

# Export US Report with Program Summary
print('[*] Exporting US report...')
try:
    with pd.ExcelWriter(output_us_file, engine='openpyxl', mode='w') as writer:
        report_us.to_excel(writer, sheet_name='USD Report US', index=False)
        if not program_summary_us.empty:
            program_summary_us.to_excel(writer, sheet_name='Program Summary', index=False)
    print(f'[*] US report exported to {output_us_file}')
    print(f'    - USD Report US sheet: {len(report_us)} rows')
    print(f'    - Program Summary sheet: {len(program_summary_us)} rows')
except Exception as e:
    print(f'[ERROR] Failed to export US report: {e}')

# Export CA Report with Program Summary
print('[*] Exporting CA report...')
try:
    with pd.ExcelWriter(output_ca_file, engine='openpyxl', mode='w') as writer:
        report_ca.to_excel(writer, sheet_name='LC Report CA', index=False)
        if not program_summary_ca.empty:
            program_summary_ca.to_excel(writer, sheet_name='Program Summary', index=False)
    print(f'[*] CA report exported to {output_ca_file}')
    print(f'    - LC Report CA sheet: {len(report_ca)} rows')
    print(f'    - Program Summary sheet: {len(program_summary_ca)} rows')
except Exception as e:
    print(f'[ERROR] Failed to export CA report: {e}')

print('\nMain reports generated successfully!')

## Individual Partner Reports Generation

In [ ]:
def create_partner_reports(df, region_name, folder_name, currency_cols):
    """Create individual partner reports"""
    print(f'[*] Creating {region_name} partner reports...')
    
    # Create output folder
    os.makedirs(folder_name, exist_ok=True)
    
    if 'Disty_Partners' not in df.columns:
        print(f'[WARNING] Disty_Partners column not found in {region_name} data')
        return
    
    # Get unique partners
    partners = df['Disty_Partners'].dropna().unique()
    print(f'[*] Found {len(partners)} unique {region_name} partners')
    
    for partner in partners:
        try:
            # Filter data for this partner
            partner_data = df[df['Disty_Partners'] == partner].copy()
            
            if len(partner_data) == 0:
                continue
            
            # Create partner program summary
            partner_program_summary = partner_data.groupby('Scheme_Name')[currency_cols].sum().reset_index()
            partner_program_summary = partner_program_summary[partner_program_summary['Scheme_Name'] != '']
            
            # Create summary report (basic aggregation)
            partner_summary = pd.DataFrame({
                'Metric': ['Total Rows', 'Unique Schemes', 'Total NDP', 'Total NET'],
                'Value': [
                    len(partner_data),
                    len(partner_program_summary),
                    partner_data[currency_cols[0]].sum() if currency_cols[0] in partner_data.columns else 0,
                    partner_data[currency_cols[1]].sum() if currency_cols[1] in partner_data.columns else 0
                ]
            })
            
            # Export to Excel file
            filename = f'{folder_name}/Disty_Partner_{int(partner)}_Report.xlsx'
            
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Main report
                partner_data.to_excel(writer, sheet_name='DSO DataSheet', index=False)
                
                # Program summary
                if not partner_program_summary.empty:
                    partner_program_summary.to_excel(writer, sheet_name='Program Summary', index=False)
                
                # Summary report
                partner_summary.to_excel(writer, sheet_name='Summary', index=False)
            
            print(f'[*] Created {region_name} Partner {int(partner)} report: {len(partner_data)} rows')
            
        except Exception as e:
            print(f'[ERROR] Failed to create {region_name} Partner {partner} report: {e}')
    
    print(f'[*] {region_name} partner reports completed')

# Generate partner reports
create_partner_reports(report_us, 'US', us_partners_folder, us_currency_cols)
create_partner_reports(report_ca, 'CA', ca_partners_folder, ca_currency_cols)

print('\nPartner reports generated successfully!')

## Results Summary

In [ ]:
print('='*60)
print('OPERATION 1 COMPLETED SUCCESSFULLY')
print('='*60)

print('\n📊 DATA PROCESSING SUMMARY:')
print(f'   • Original data rows: {len(df):,}')
print(f'   • US final rows: {len(report_us):,}')
print(f'   • CA final rows: {len(report_ca):,}')
print(f'   • RCS reference rows: {len(df_rcs):,}')

print('\n📁 GENERATED FILES:')
print(f'   • {output_us_file} (US main report)')
print(f'   • {output_ca_file} (CA main report)')
print(f'   • {us_partners_folder}/ (US partner reports)')
print(f'   • {ca_partners_folder}/ (CA partner reports)')

print('\n📈 PROGRAM SUMMARIES:')
print(f'   • US schemes: {len(program_summary_us)}')
print(f'   • CA schemes: {len(program_summary_ca)}')

if 'Disty_Partners' in report_us.columns:
    us_partners_count = len(report_us['Disty_Partners'].dropna().unique())
    print(f'   • US partners: {us_partners_count}')

if 'Disty_Partners' in report_ca.columns:
    ca_partners_count = len(report_ca['Disty_Partners'].dropna().unique())
    print(f'   • CA partners: {ca_partners_count}')

print('\n✅ All Operation 1 processing and report generation completed!')
print('\n💡 Next steps:')
print('   • Review generated Excel files')
print('   • Verify data accuracy in reports')
print('   • Check partner-specific reports in folders')
print('='*60)